In [1]:
import io, os, sys, types, time, datetime, math, random, requests, subprocess,io, tempfile

In [2]:
# 数据处理
import numpy as np
import pandas as pd
from scipy.stats import chi2
import numba as nb

# 数据可视化
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
from pyecharts import options as opts
from pyecharts.charts import Pie
from mpl_toolkits.mplot3d import Axes3D

# 特征选择和编码
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# 机器学习
import sklearn.ensemble as ske
from sklearn import datasets, model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# 网格搜索、随机搜索
import scipy.stats as st
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# 模型度量（分类）
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# 警告处理 
import warnings
warnings.filterwarnings('ignore')

# 在Jupyter上画图
%matplotlib inline

In [3]:
data = pd.read_csv(r"/home/mw/input/credit_card8849/card_transdata.csv")
data.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0



## 字段说明：  
#### distance_from_home:离发生交易的位置的距离  
#### distance_from_transaction:距离上一次交易发生的距离  
#### ratio_to_median_purchase_price:购买价格与中位购买价格的比值  
#### repeat_retailer：交易是否来自同一零售商  
#### uesd_pin_number:交易是否使用密码进行  
#### online_order:交易是否为在线订单  
#### fraud:交易是否具有欺诈性  
#### fraud=1表示交易存在欺诈性

In [4]:
data.sample(20)

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
715949,46.973854,0.047134,2.142675,1.0,0.0,0.0,0.0,0.0
718284,31.447046,0.210591,0.648232,1.0,0.0,0.0,0.0,0.0
614201,35.258404,0.947318,1.429857,1.0,0.0,0.0,0.0,0.0
245168,154.378197,2.145163,0.895301,1.0,1.0,0.0,0.0,0.0
313788,28.776243,2.131254,0.487957,1.0,0.0,0.0,1.0,0.0
732590,6.148063,0.891077,1.795119,1.0,1.0,1.0,1.0,0.0
512134,48.797335,1.216731,1.875073,1.0,1.0,0.0,0.0,0.0
105465,7.250681,5.526212,0.786724,1.0,0.0,0.0,1.0,0.0
45542,16.136890,4.791327,0.410421,1.0,1.0,0.0,0.0,0.0
765165,16.360440,1.875691,1.296199,1.0,0.0,0.0,1.0,0.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
distance_from_home                1000000 non-null float64
distance_from_last_transaction    1000000 non-null float64
ratio_to_median_purchase_price    1000000 non-null float64
repeat_retailer                   1000000 non-null float64
used_chip                         1000000 non-null float64
used_pin_number                   1000000 non-null float64
online_order                      1000000 non-null float64
fraud                             1000000 non-null float64
dtypes: float64(8)
memory usage: 61.0 MB


In [6]:
@nb.jit(nopython=True)
def check_duplicates(data, columns):
    return data.duplicated(columns)
duplicates = data.duplicated(['distance_from_home','distance_from_last_transaction','ratio_to_median_purchase_price','repeat_retailer','used_chip','used_pin_number','online_order','fraud'])
true_count = duplicates.sum()
print("重复行的数量：", true_count)

重复行的数量： 0


## 从输出结果来看，没有重复值

In [7]:
missingno.matrix(data, figsize = (30,5))

<Figure size 2160x360 with 2 Axes>

## 没有缺失值  
## 评估无效或错误数据

In [8]:
data.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,26.628792,5.036519,1.824182,0.881536,0.350399,0.100608,0.650552,0.087403
std,65.390784,25.843093,2.799589,0.323157,0.477095,0.300809,0.476796,0.282425
min,0.004874,0.000118,0.004399,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.878008,0.296671,0.475673,1.000000,0.000000,0.000000,0.000000,0.000000
50%,9.967760,0.998650,0.997717,1.000000,0.000000,0.000000,1.000000,0.000000
75%,25.743985,3.355748,2.096370,1.000000,1.000000,0.000000,1.000000,0.000000
max,10632.723672,11851.104565,267.802942,1.000000,1.000000,1.000000,1.000000,1.000000


## 由输出结果来看，不存在无效和错误数据

In [9]:
# 绘制每个特征的分布
def plot_distribution(data, cols=5, width=20, height=15, hspace=0.2, wspace=0.5):
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure(figsize=(width,height))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=wspace, hspace=hspace)
    rows = math.ceil(float(data.shape[1]) / cols)
    for i, column in enumerate(data.columns):
        ax = fig.add_subplot(rows, cols, i + 1)
        ax.set_title(column)
        
        if data.dtypes[column] == np.object:
            g = sns.countplot(y=column, data=data)
            substrings = [s.get_text()[:18] for s in g.get_yticklabels()]
            g.set(yticklabels=substrings)
            plt.xticks(rotation=25)
        
        else:
            g = sns.distplot(data[column])
            plt.xticks(rotation=25)
    
plot_distribution(data, cols=3, width=20, height=20, hspace=0.45, wspace=0.5)

<Figure size 1440x1440 with 8 Axes>

In [10]:
plt.rcParams['font.sans-serif']=['Microsoft YaHei']
plt.rcParams['axes.unicode_minus']=False

In [11]:
fraud_distribution = data['fraud'].value_counts()
print(fraud_distribution)

0.0    912597
1.0     87403
Name: fraud, dtype: int64


In [12]:
from pyecharts import options as opts
from pyecharts.charts import Bar
fraud_counts = [912597, 87403]
fraud_labels = ['非欺诈', '欺诈']
bar = (
    Bar()
    .add_xaxis(fraud_labels)
    .add_yaxis("", fraud_counts, itemstyle_opts=opts.ItemStyleOpts())
    .set_global_opts(title_opts=opts.TitleOpts(title="欺诈分布情况 (0: 非欺诈, 1: 欺诈)"))
    .set_series_opts(label_opts=opts.LabelOpts(position="top"))
)
bar.render_notebook()

In [13]:
fraud_counts = [912597, 87403]
fraud_labels = ['非欺诈', '欺诈']
colors = ['#FF6347', '#4682B4']
# 创建饼图
pie = (
    Pie()
    .add("", [list(z) for z in zip(fraud_labels, fraud_counts)], center=["50%", "50%"])
    .set_colors(colors)
    .set_global_opts(title_opts=opts.TitleOpts(title="欺诈分布情况 (0: 非欺诈, 1: 欺诈)"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
pie.render_notebook()

In [14]:
# 生成热力图，显示数据集中各个变量之间的相关性
correlation_matrix=data.corr()
import seaborn as sns
sns.heatmap(correlation_matrix,annot=True,cmap='coolwarm',linewidths=0.5)

<Figure size 432x288 with 2 Axes>

In [15]:
# 使用随机森林算法去查看属性重要性
clf = RandomForestClassifier()
clf.fit(data.drop('fraud', axis=1), data['fraud'])
plt.style.use('seaborn-whitegrid')
importance = clf.feature_importances_
importance = pd.DataFrame(importance, index=data.drop('fraud', axis=1).columns, columns=["Importance"])
importance.sort_values(by='Importance', ascending=True).plot(kind='barh', figsize=(20,len(importance)/2));

<Figure size 1440x252 with 1 Axes>

## 从图表中可以看出，“distance_from_home”是最重要的特征，这意味着消费者更倾向于在离他们家较近的地方进行信用卡交易。这可能与消费者的便利性和安全性有关，因为离家近的地方更容易访问，并且消费者可能认为在这些地方进行交易更安全。  

## “used_chip”和“repeat_retailer”也是重要的特征，这表明消费者更倾向于使用芯片进行交易，并且更愿意重复光顾同一个零售商。这可能与芯片交易的安全性和消费者对特定零售商的信任度有关。  

## 相比之下，“online_order”和“used_pin_number”的重要性相对较低，这可能意味着消费者对在线订单和个人识别号码的安全性认识不足，或者他们认为这不是决定信用卡交易的主要因素。  

## 通过分析这些特征的重要性，我们可以更好地理解消费者的购物行为和偏好，并利用这些信息来预测信用卡诈骗。例如，如果一个交易发生在离家很远的地方或涉及到不寻常的购买行为（如频繁使用个人识别号码），那么这可能是一个潜在的诈骗信号，需要进一步调查。

In [16]:
# 通过使用了SMOTE算法来对数据进行过采样，以平衡类别分布。
X = data.iloc[:, :-1]
y = data.iloc[:, -1]  
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

In [17]:
# 计算经过重采样后的数据集中样本的个数以及类别0和类别1的比例
n_sample_new = X_resampled.shape[0]
value_counts = y_resampled.value_counts()
n_0_sample_new = value_counts[0]
n_1_sample_new = value_counts[1]

percentage_0 = (n_0_sample_new / n_sample_new) * 100
percentage_1 = (n_1_sample_new / n_sample_new) * 100

print(f'样本个数: {n_sample_new}; 0占{percentage_0:.2f}%, 1占{percentage_1:.2f}%')

样本个数: 1825194; 0占50.00%, 1占50.00%


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_resampled,y_resampled,test_size=0.2,random_state=888)

In [19]:
X_train.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order
110633,95.575519,0.118122,0.678609,1.0,0.0,0.0,1.0
478984,5.020114,1.928217,0.745472,1.0,0.0,0.0,0.0
443406,11.297672,18.806170,0.787130,1.0,0.0,0.0,1.0
336915,35.454609,2.490169,2.643938,1.0,0.0,0.0,0.0
912834,5.140378,1.573739,1.850330,1.0,1.0,0.0,0.0


In [20]:
y_train.head()

110633    0.0
478984    0.0
443406    0.0
336915    0.0
912834    0.0
Name: fraud, dtype: float64

In [21]:
# 绘制ROC曲线
def plot_roc_curve(y_test, preds):
    fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
    roc_auc = metrics.auc(fpr, tpr)
    plt.title('ROC')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.ylabel('TPR')
    plt.xlabel('FPR')
    plt.show()

In [22]:
# 训练机器学习算法并输出预测结果
def fit_ml_algo(algo, X_train, y_train, X_test, cv):
    # 训练
    model = algo.fit(X_train, y_train)
    # 预测
    test_pred = model.predict(X_test)
    if (isinstance(algo, (LogisticRegression, 
                          KNeighborsClassifier, 
                          GaussianNB, 
                          DecisionTreeClassifier, 
                          RandomForestClassifier,
                          GradientBoostingClassifier))):
        probs = model.predict_proba(X_test)[:,1]
    else:
        probs = "Not Available"
    acc = round(model.score(X_test, y_test) * 100, 2) 
    # 交叉验证
    train_pred = model_selection.cross_val_predict(algo, 
                                                  X_train, 
                                                  y_train, 
                                                  cv=cv, 
                                                  n_jobs = -1) # 所有CPU核
    acc_cv = round(metrics.accuracy_score(y_train, train_pred) * 100, 2)
    return train_pred, test_pred, acc, acc_cv, probs

In [23]:
# 逻辑回归（随机选择最优参数）
# 输出最佳参数的统一方法
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
# 设置参数
param_dist = {'penalty': ['l2', 'l1'], 
                         'class_weight': [None, 'balanced'],
                         'C': np.logspace(-20, 20, 10000), 
                         'intercept_scaling': np.logspace(-20, 20, 10000)}

# 随机选取参数
n_iter_search = 15
lrc = LogisticRegression()
random_search = RandomizedSearchCV(lrc, 
                                   n_jobs=-1, 
                                   param_distributions=param_dist, 
                                   n_iter=n_iter_search)

start = time.time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 872.62 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.941 (std: 0.001)
Parameters: {'penalty': 'l2', 'intercept_scaling': 4.2546406611917926e-12, 'class_weight': None, 'C': 9183247719.403553}

Model with rank: 2
Mean validation score: 0.941 (std: 0.001)
Parameters: {'penalty': 'l2', 'intercept_scaling': 2.053199941072397e-16, 'class_weight': 'balanced', 'C': 9.249038977451963}

Model with rank: 3
Mean validation score: 0.941 (std: 0.001)
Parameters: {'penalty': 'l2', 'intercept_scaling': 0.0011109757270911643, 'class_weight': None, 'C': 2903663561.7833357}

Model with rank: 4
Mean validation score: 0.941 (std: 0.001)
Parameters: {'penalty': 'l2', 'intercept_scaling': 1.7870027157812014e-19, 'class_weight': 'balanced', 'C': 2.1673726731560632e+19}

Model with rank: 5
Mean validation score: 0.879 (std: 0.000)
Parameters: {'penalty': 'l2', 'intercept_scaling': 82128.03824272337, 'class_weight': None, 'C': 7.471147983930855

In [24]:
# 训练
start_time = time.time()
train_pred_log, test_pred_log, acc_log, acc_cv_log, probs_log = fit_ml_algo(LogisticRegression(n_jobs = -1), 
                                                                 X_train, 
                                                                 y_train, 
                                                                 X_test, 
                                                                 10)
log_time = (time.time() - start_time)
print("Accuracy: %s" % acc_log)
print("Accuracy CV 10-Fold: %s" % acc_cv_log)
print("Running Time: %s" % datetime.timedelta(seconds=log_time))

Accuracy: 94.04
Accuracy CV 10-Fold: 94.1
Running Time: 0:07:21.293099


In [25]:
print(metrics.classification_report(y_train, train_pred_log) )

              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94    729991
         1.0       0.93      0.95      0.94    730164

    accuracy                           0.94   1460155
   macro avg       0.94      0.94      0.94   1460155
weighted avg       0.94      0.94      0.94   1460155



In [26]:
plot_roc_curve(y_test, probs_log)

<Figure size 432x288 with 1 Axes>

In [27]:
# KNN
start_time = time.time()
train_pred_knn, test_pred_knn, acc_knn, acc_cv_knn, probs_knn = fit_ml_algo(KNeighborsClassifier(n_neighbors = 5,
                                                                                                 n_jobs = -1), 
                                                                                                 X_train, 
                                                                                                 y_train, 
                                                                                                 X_test, 
                                                                                                 10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy CV 10-Fold: %s" % acc_cv_knn)
print("Running Time: %s" % datetime.timedelta(seconds=knn_time))

Accuracy: 98.71
Accuracy CV 10-Fold: 98.63
Running Time: 0:02:22.694846


In [28]:
print (metrics.classification_report(y_train, train_pred_knn) )

              precision    recall  f1-score   support

         0.0       1.00      0.97      0.99    729991
         1.0       0.97      1.00      0.99    730164

    accuracy                           0.99   1460155
   macro avg       0.99      0.99      0.99   1460155
weighted avg       0.99      0.99      0.99   1460155



In [29]:
print (metrics.classification_report(y_test, test_pred_knn) )

              precision    recall  f1-score   support

         0.0       1.00      0.97      0.99    182606
         1.0       0.98      1.00      0.99    182433

    accuracy                           0.99    365039
   macro avg       0.99      0.99      0.99    365039
weighted avg       0.99      0.99      0.99    365039



In [30]:
plot_roc_curve(y_test, probs_knn)

<Figure size 432x288 with 1 Axes>

In [31]:
# 高斯朴素贝叶斯
start_time = time.time()
train_pred_gaussian, test_pred_gaussian, acc_gaussian, acc_cv_gaussian, probs_gau = fit_ml_algo(GaussianNB(), 
                                                                                     X_train, 
                                                                                     y_train, 
                                                                                     X_test, 
                                                                                     10)
gaussian_time = (time.time() - start_time)
print("Accuracy: %s" % acc_gaussian)
print("Accuracy CV 10-Fold: %s" % acc_cv_gaussian)
print("Running Time: %s" % datetime.timedelta(seconds=gaussian_time))

Accuracy: 78.58
Accuracy CV 10-Fold: 79.27
Running Time: 0:00:04.680256


In [32]:
print (metrics.classification_report(y_train, train_pred_gaussian) )

              precision    recall  f1-score   support

         0.0       0.95      0.62      0.75    729991
         1.0       0.72      0.96      0.82    730164

    accuracy                           0.79   1460155
   macro avg       0.83      0.79      0.79   1460155
weighted avg       0.83      0.79      0.79   1460155



In [33]:
print( metrics.classification_report(y_test, test_pred_gaussian) )

              precision    recall  f1-score   support

         0.0       0.95      0.61      0.74    182606
         1.0       0.71      0.96      0.82    182433

    accuracy                           0.79    365039
   macro avg       0.83      0.79      0.78    365039
weighted avg       0.83      0.79      0.78    365039



In [34]:
plot_roc_curve(y_test, probs_gau)

<Figure size 432x288 with 1 Axes>

In [35]:
# 支持向量机算法
start_time = time.time()
train_pred_svc, test_pred_svc, acc_linear_svc, acc_cv_linear_svc, _ = fit_ml_algo(LinearSVC(),
                                                                                           X_train, 
                                                                                           y_train,
                                                                                           X_test, 
                                                                                           10)
linear_svc_time = (time.time() - start_time)
print("Accuracy: %s" % acc_linear_svc)
print("Accuracy CV 10-Fold: %s" % acc_cv_linear_svc)
print("Running Time: %s" % datetime.timedelta(seconds=linear_svc_time))

Accuracy: 94.2
Accuracy CV 10-Fold: 92.7
Running Time: 0:35:14.909660


In [36]:
print (metrics.classification_report(y_train, train_pred_svc) )

              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93    729991
         1.0       0.93      0.93      0.93    730164

    accuracy                           0.93   1460155
   macro avg       0.93      0.93      0.93   1460155
weighted avg       0.93      0.93      0.93   1460155



In [37]:
print (metrics.classification_report(y_test, test_pred_svc)) 

              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94    182606
         1.0       0.93      0.95      0.94    182433

    accuracy                           0.94    365039
   macro avg       0.94      0.94      0.94    365039
weighted avg       0.94      0.94      0.94    365039



In [38]:
# 决策树
start_time = time.time()
train_pred_dt, test_pred_dt, acc_dt, acc_cv_dt, probs_dt = fit_ml_algo(DecisionTreeClassifier(), 
                                                             X_train, 
                                                             y_train, 
                                                             X_test, 
                                                             10)
dt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_dt)
print("Accuracy CV 10-Fold: %s" % acc_cv_dt)
print("Running Time: %s" % datetime.timedelta(seconds=dt_time))

Accuracy: 100.0
Accuracy CV 10-Fold: 100.0
Running Time: 0:00:55.965971


In [39]:
print (metrics.classification_report(y_train, train_pred_dt) )

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    729991
         1.0       1.00      1.00      1.00    730164

    accuracy                           1.00   1460155
   macro avg       1.00      1.00      1.00   1460155
weighted avg       1.00      1.00      1.00   1460155



In [40]:
print (metrics.classification_report(y_test, test_pred_dt) )

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    182606
         1.0       1.00      1.00      1.00    182433

    accuracy                           1.00    365039
   macro avg       1.00      1.00      1.00    365039
weighted avg       1.00      1.00      1.00    365039



In [41]:
plot_roc_curve(y_test, probs_dt)

<Figure size 432x288 with 1 Axes>

In [42]:
# 随机森林

# 输出参数得分
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
# 设置参数范围
param_dist = {"max_depth": [4, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 10),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# 随机搜索
n_iter_search = 10
rfc = RandomForestClassifier(n_estimators=4)
random_search = RandomizedSearchCV(rfc, 
                                   n_jobs = -1, 
                                   param_distributions=param_dist, 
                                   n_iter=n_iter_search)

start = time.time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 147.96 seconds for 10 candidates parameter settings.
Model with rank: 1
Mean validation score: 1.000 (std: 0.000)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 3, 'min_samples_leaf': 8, 'min_samples_split': 8}

Model with rank: 2
Mean validation score: 1.000 (std: 0.000)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 1, 'min_samples_leaf': 10, 'min_samples_split': 9}

Model with rank: 3
Mean validation score: 0.996 (std: 0.000)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 5, 'min_samples_split': 4}

Model with rank: 4
Mean validation score: 0.944 (std: 0.068)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 7, 'min_samples_split': 7}

Model with rank: 5
Mean validation score: nan (std: nan)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_dep

In [43]:
# 训练
start_time = time.time()
rfc = RandomForestClassifier(n_estimators=4, 
                             min_samples_leaf=1,
                             min_samples_split=3, 
                             criterion='gini', 
                             max_features=X_train.shape[1] // 2)

train_pred_rf, test_pred_rf, acc_rf, acc_cv_rf, probs_rf = fit_ml_algo(rfc, 
                                                             X_train, 
                                                             y_train, 
                                                             X_test, 
                                                             6)
rf_time = (time.time() - start_time)
print("Accuracy: %s" % acc_rf)
print("Accuracy CV 10-Fold: %s" % acc_cv_rf)
print("Running Time: %s" % datetime.timedelta(seconds=rf_time))

Accuracy: 100.0
Accuracy CV 10-Fold: 100.0
Running Time: 0:00:54.343644


In [44]:
from sklearn.metrics import accuracy_score
train_score = []
test_score = []

for i in range(1, 10):
    rf = RandomForestClassifier(n_estimators=100, max_depth=i, random_state=66)  # 随机森林模型设置

    model_rf = rf.fit(X_train, y_train)
    train_score.append(accuracy_score(model_rf.predict(X_train), y_train))
    test_score.append(accuracy_score(model_rf.predict(X_test), y_test))

# 绘制图表
plt.figure(figsize=(12, 6))
for i, (score, color, label) in enumerate(zip([train_score, test_score], ["green", "black"], ["train_score", "test_score"])):
    plt.plot(range(1, 10), score, color=color, label=label)

plt.legend(loc='lower right')  # 图例的位置在右下角
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy vs. Max Depth')
plt.show()

<Figure size 864x432 with 1 Axes>

In [45]:
print(metrics.classification_report(y_train, train_pred_rf) )

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    729991
         1.0       1.00      1.00      1.00    730164

    accuracy                           1.00   1460155
   macro avg       1.00      1.00      1.00   1460155
weighted avg       1.00      1.00      1.00   1460155



In [46]:
print( metrics.classification_report(y_test, test_pred_rf) )

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    182606
         1.0       1.00      1.00      1.00    182433

    accuracy                           1.00    365039
   macro avg       1.00      1.00      1.00    365039
weighted avg       1.00      1.00      1.00    365039



In [47]:
plot_roc_curve(y_test, probs_rf)

<Figure size 432x288 with 1 Axes>

In [48]:
# 梯度提升树
start_time = time.time()
train_pred_gbt, test_pred_gbt, acc_gbt, acc_cv_gbt, probs_gbt = fit_ml_algo(GradientBoostingClassifier(), 
                                                                 X_train, 
                                                                 y_train, 
                                                                 X_test, 
                                                                 10)
gbt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_gbt)
print("Accuracy CV 10-Fold: %s" % acc_cv_gbt)
print("Running Time: %s" % datetime.timedelta(seconds=gbt_time))

Accuracy: 99.94
Accuracy CV 10-Fold: 99.95
Running Time: 0:39:21.466305


In [49]:
print(metrics.classification_report(y_train, train_pred_gbt) )

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    729991
         1.0       1.00      1.00      1.00    730164

    accuracy                           1.00   1460155
   macro avg       1.00      1.00      1.00   1460155
weighted avg       1.00      1.00      1.00   1460155



In [50]:
print( metrics.classification_report(y_test, test_pred_gbt) )

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    182606
         1.0       1.00      1.00      1.00    182433

    accuracy                           1.00    365039
   macro avg       1.00      1.00      1.00    365039
weighted avg       1.00      1.00      1.00    365039



In [51]:
plot_roc_curve(y_test, probs_gbt)

<Figure size 432x288 with 1 Axes>

In [52]:
models = pd.DataFrame({
    'Model': ['KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 
              'Linear SVC', 
              'Decision Tree', 'Gradient Boosting Trees'],
    'Score': [
        acc_cv_knn, 
        acc_cv_log,     
        acc_cv_rf, 
        acc_cv_gaussian, 
        acc_cv_linear_svc, 
        acc_cv_dt,
        acc_cv_gbt
    ]})
models.sort_values(by='Score', ascending=False)

,Model,Score
2,Random Forest,100.00
5,Decision Tree,100.00
6,Gradient Boosting Trees,99.95
0,KNN,98.63
1,Logistic Regression,94.10
4,Linear SVC,92.70
3,Naive Bayes,79.27


In [53]:
plt.style.use('seaborn-whitegrid')
fig = plt.figure(figsize=(6,6)) 

models = [
    'KNN', 
    'Logistic Regression', 
    'Random Forest', 
    'Naive Bayes', 
    'Decision Tree', 
    'Gradient Boosting Trees'
]
probs = [
    probs_knn,
    probs_log,
    probs_rf,
    probs_gau,
    probs_dt,
    probs_gbt
]
colors = [
    'blue',
    'green',
    'red',
    'cyan',
    'magenta',
    'yellow',
]
    
plt.title('Receiver Operating Characteristic')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

def plot_roc_curves(y_test, prob, model):
    fpr, tpr, threshold = metrics.roc_curve(y_test, prob)
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, 'b', label = model + ' AUC = %0.2f' % roc_auc, color=colors[i])
    plt.legend(loc = 'lower right')
    
for i, model in list(enumerate(models)):
    plot_roc_curves(y_test, probs[i], models[i])
    
plt.show()

<Figure size 432x432 with 1 Axes>